In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import time
import re
import os

with open('../Secret_file.json',"r") as f:
    ks_token = json.load(f)['Slack_API']['KS']

In [ ]:
'''
時間阻斷器
'''
def time_start(hour):
    # 時
    while True:
        if datetime.now().hour == (hour-1):
            break
        else:
            time.sleep(3600)
            continue

    # 分
    while True:
        if datetime.now().minute == 59:
            break
        else:
            time.sleep(60)
            continue

    # 秒
    while True:
        if datetime.now().second >= 50:
            break
        else:
            time.sleep(5)
            continue

    while True:
        if datetime.now().hour == hour:
            break
        else:
            continue
time_start(2)
print('執行時間：\n', datetime.now())

In [ ]:
def parse_time(times):
    if times == 0:
        return 'No data'
    else:
        t = time.localtime(times)
        return time.strftime('%Y/%m/%d', t)

def watches(project_url):
    url = project_url
    time.sleep(0.5)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html5lib')
    token = soup.find('meta', attrs={'name':'csrf-token'})['content']
    cookies = r.cookies['_ksr_session']
    
    url = 'https://www.kickstarter.com/graph'
    headers = {'content-type':'application/json',
               'cookie':'_ksr_session={}'.format(cookies),
               'x-csrf-token':'{}'.format(token)}
    key_word_list = project_url.split('/')
    payloads = {"operationName":"PrelaunchPage",
                "variables":{"slug":"{}/{}".format(key_word_list[-2], key_word_list[-1])},
                "query":"query PrelaunchPage($slug: String!) {\n  me {\n    id\n    isKsrAdmin\n    __typename\n  }\n  project(slug: $slug) {\n    id\n    name\n    location {\n      discoverUrl\n      displayableName\n      countryName\n      __typename\n    }\n    state\n    description\n    url\n    imageUrl(width: 1000)\n    prelaunchActivated\n    isWatched\n    watchesCount\n    category {\n      url\n      name\n      parentCategory {\n        name\n        __typename\n      }\n      __typename\n    }\n    verifiedIdentity\n    creator {\n      id\n      name\n      slug\n      hasImage\n      imageUrl(width: 48)\n      url\n      biography\n      backingsCount\n      isFacebookConnected\n      lastLogin\n      websites {\n        url\n        domain\n        __typename\n      }\n      launchedProjects {\n        totalCount\n        __typename\n      }\n      location {\n        displayableName\n        __typename\n      }\n      __typename\n    }\n    collaborators {\n      edges {\n        node {\n          name\n          url\n          imageUrl(width: 48)\n          __typename\n        }\n        title\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n"}
    rp = requests.post(url, data=json.dumps(payloads), headers=headers)
    jsdata = json.loads(rp.text)
    return jsdata['data']['project']['watchesCount']

def get_digit(target):
    num = ''
    for x in target:
        try:
            if x.isdigit():
                num += x
        except:
            pass
    return int(num)

def Bot_Message(channel, text):
    payload={
        "token":ks_token,
        "channel":channel,
        "text":'{}'.format(text)} 
    r = requests.post("	https://slack.com/api/chat.postMessage", params=payload) 

check_point = 0
while check_point <5:
    try:
        start_time = time.time()
        '''
        today total list
        '''
        url = 'https://www.kickstarter.com/discover/advanced?state=upcoming&category_id=34&sort=magic&seed=2644890&page=1'

        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html5lib')

        total = soup.find(class_='count ksr-green-500').text
        total = get_digit(total)

        #one page for 12 projects
        if total%12 != 0:
            total = (total //12)+1
        else:
            total = total//12


        '''
        get information for today list
        '''
        project_list = []
        for x in range(1, total+1):
            #注意seed不要做更動，否則會造成重複爬取
            list_url = 'https://www.kickstarter.com/discover/advanced?state=upcoming&category_id=34&sort=magic&seed=2644890&page='+str(x)
            time.sleep(1)
            r = requests.get(list_url)
            soup = BeautifulSoup(r.text, 'html5lib')


            for y in soup.find_all(class_='js-react-proj-card grid-col-12 grid-col-6-sm grid-col-4-lg'):
                data = json.loads(y['data-project'])
                
                temp = []
                temp.append(data['name'])
                temp.append(data['goal'])
                temp.append(data['country'])
                temp.append(parse_time(data['created_at']))
                temp.append(parse_time(data['state_changed_at']))
                temp.append(parse_time(data['deadline']))
                temp.append(data['creator']['name'])
                project_url = data['urls']['web']['project']
                temp.append(project_url)
                temp.append(watches(project_url))

                project_list.append(temp)

        str_time = time.strftime('%y%m%d', time.localtime(time.time()))
        df = pd.DataFrame(project_list, columns=['name', 'goal', 'country', 'creat_time', 'state_changed', 'deadline', 'creator', 'project_url', 'Watches_{}'.format(str_time)])

        if os.path.exists('../../save_data/KS/KS_project_upcoming.csv'):
            data = pd.read_csv('../../save_data/KS/KS_project_upcoming.csv')
        else:
            df.to_csv('../../save_data/KS/KS_project_upcoming.csv', index=False)
            time.sleep(86400-(time.time()-start_time)) #1 【計時器】
            continue
        '''
        check complete project
        and output
        '''
        pass_pro = []
        for i, project_name in enumerate(data['name']):
            if project_name not in list(df['name']):
                pass_pro.append((i, project_name, data.iloc[i, -1]))
            else:
                continue
        message = '*【Started projects】*\n{}'.format(('\n').join('• '+x+'\nFollows：'+str(y) for i,x,y in pass_pro))
        Bot_Message('kickstarter_news', message)

        for i, pro, watch in pass_pro:
            pass_data = pd.DataFrame([data.loc[i, :]]).dropna(axis=1)
            startt = pass_data.columns[8][-6:]
            endt = pass_data.columns[-1][-6:]
            pro = re.sub('[\.\"\\\/\:\*\?\<\>\|]+', '_', pro) #for file name
            try:
                pass_data.to_csv('../../save_data/KS/KS_upcoming_list/{}~{}_{}.csv'.format(startt, endt, pro), index=False)
            except:
                Bot_Message('kickstarter_news', 'Wrong file name with :',pro)

            data.drop(index=i, inplace=True)

        data.reset_index(inplace=True, drop=True)


        '''
        check new project
        '''
        new_pro = []
        for project_name in df['name']:
            if project_name not in list(data['name']):
                new_pro.append(project_name)
            else:
                continue
        message = '*【New projects】*\n{}'.format('\n'.join('• '+x for x in new_pro))
        Bot_Message('kickstarter_news', message)


        '''
        Insert data
        '''
        #data update
        df1 = df[~df.name.isin(new_pro)] #old
        df2 = df[df.name.isin(new_pro)] #new
        data = pd.merge(data, df1.iloc[:, -2:], on='project_url')
        data = pd.concat([data, df2], axis=0, ignore_index=True, sort=False)
        data.reset_index(inplace=True, drop=True)
        data.dropna(axis=1, how='all', inplace=True)
        data.to_csv('../../save_data/KS/KS_project_upcoming.csv', index=False)
        
        check_point = 0 #2 【偵錯器】
        time.sleep(86400-(time.time()-start_time)) #1 【計時器】
                    
    except:
        check_point += 1 #2 【偵錯器】
        Bot_Message('spider_status', '<@D.E.>\nGet ERROR for KS list !({}/{})\nPlease check the codes or website .'.format(check_point, 5))
        time.sleep(5)

----